Test the grit measurement on a articifial loss landscape.

In [1]:
import torch
torch.set_grad_enabled(False)
torch.set_default_dtype(torch.double)

x = torch.tensor([[0.0], [1.0]])  # Inputs in double precision
y = torch.tensor([[0.0], [1.0]])  # Targets in double precision

model = torch.nn.Linear(1, 1)  # Model initialized and converted to double precision
criterion = torch.nn.functional.mse_loss  # Loss function

EPSILON = 1e-12

l = []

l.append(float(criterion(model(x), y)))

for i in range(2):
    # modify the params
    for param in model.parameters():
        param += EPSILON
    l.append(float(criterion(model(x), y)))

for j in l:
    print(j)

print(l[0]-2*l[1]+l[2])


0.044088292117439996
0.044088292118041855
0.044088292118643665
-4.85722573273506e-17


In [2]:
# custom modules
from utils.profiler import profiler
from utils.param_math import iadd, scale, sub, norm, normalize, rand_like, project_to_module
from workflow import directions, dirs_and_dists, dists_to_models, eval_ensemble, curvature_scale_analysis, plot

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# # Generate synthetic data
# x = torch.rand(100, 1)  # 100 data points between 0 and 1
# K = 1
# D = 0.5
# noise = torch.randn(100, 1) * 0.2
# y = K * x + D + noise
# Define the dataset with two points (0,0) and (1,1)
x = torch.tensor([[0.0], [1.0]], dtype=torch.double)  # Inputs in double precision
y = torch.tensor([[0.0], [1.0]], dtype=torch.double)  # Targets in double precision


Running on cpu

:::::::::::::::1 step of gradient descent over the whole dataset::::::::::::::::
ram used :    4.8 | ram avail:    6.2 | 
ram used :   +0.0 | ram avail:   -0.0 | 
:::::::::22.66 s for 1 step of gradient descent over the whole dataset::::::::::

-------------------------------Ensemble size: 65--------------------------------
ram used :    4.9 | ram avail:    6.1 | 

:::::::::::::::::::::::::::::::Stacking Ensemble::::::::::::::::::::::::::::::::
ram used :    4.9 | ram avail:    6.1 | 
ram used :   +0.1 | ram avail:   -0.1 | 
::::::::::::::::::::::::::0.05 s for Stacking Ensemble::::::::::::::::::::::::::

:::::::::::::::::::::::::::::Deleting Ensemble List:::::::::::::::::::::::::::::
ram used :    5.0 | ram avail:    6.1 | 
ram used :   +0.0 | ram avail:   +0.0 | 
:::::::::::::::::::::::0.00 s for Deleting Ensemble List::::::::::::::::::::::::

:::::::::::::::::::::::Evaluating stacked ensemble om cpu:::::::::::::::::::::::
ram used :    5.0 | ram avail:    6.1 | 


KeyboardInterrupt: 

In [ ]:

torch.set_grad_enabled(False)

# config dict for the experiment
c = {
    'device': "cpu",
    'center': torch.nn.Linear(1, 1).double(),   # the point in parameter space to start from
    'dataloader': DataLoader(TensorDataset(x, y)),
    'criterion': torch.nn.functional.mse_loss,
    'grad': False,                        # should we look in the direction of gradient ascent and descent?
    'subspaces': [], # project directions onto modules with these substrings in their names. See project_to_module()
    'rand_dirs': 3,     # number of random directions to add, in addition to the gradient and radial directions
    'max_oom': 2,     # furthest sample will be 10**max_oom from the center
    'min_oom':-13,    # closest sample will be 10**min_oom from the center
}

print(f'Running on {c["device"]}')


dirs = directions(c)

df = dirs_and_dists(dirs.keys(), c['min_oom'], c['max_oom'])

df = dists_to_models(df, dirs, c['center'])

# add the center model
ensemble_list = [c['center']] + list(df['Model'])
df.drop(columns='Model', inplace=True)


ensemble_loss = eval_ensemble(ensemble_list, c['dataloader'], c['criterion'], c['device'])

# Convert the loss to a list and unpack
center_loss, *df['Loss'] = ensemble_loss.tolist()

# Add the center loss to the DataFrame at 'Distance' = 0
for direction in df.index.get_level_values('Direction'):
    df.loc[(direction, 0), ['Distance', 'Loss']] = [0., center_loss]

df.sort_index(inplace=True)

# round df['Loss'] to 1 decimal place
# df['Loss'] = df['Loss'].round(8)

df = curvature_scale_analysis(df)

df.head()

figs = plot(df, 'Linear Regression')
for fig in figs:
    fig.show()

Running on cpu

:::::::::::::::1 step of gradient descent over the whole dataset::::::::::::::::
ram used :    6.1 | ram avail:    4.7 | 
ram used :   +0.0 | ram avail:   +0.0 | 
::::::::::0.00 s for 1 step of gradient descent over the whole dataset::::::::::


RuntimeError: Double did not match Float